In [1]:
!pip install diffusers==0.10.2 transformers scipy ftfy accelerate
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install gradio


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.1/503.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-th5jdrjn
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-th5jdrjn
  Re

In [2]:
from PIL import Image
import requests
import torch
from io import BytesIO

from diffusers import StableDiffusionInpaintPipeline

from segment_anything import SamPredictor, sam_model_registry

import gradio as gr

import numpy as np

## Todo: Initiate SAM predictor

In [3]:
device = 'cuda' if torch.cuda.is_available() else "cpu"
sam_checkpoint = '/content/drive/MyDrive/Colab_Notebooks/DiffusionModel/Sam_StableDiffusion/weights/sam_vit_h_4b8939.pth'
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

## Authenticate with Hugging Face to access the Stable Diffusion model

In [4]:
from huggingface_hub import notebook_login

# Required to get access to stable diffusion model
notebook_login()

## # Todo: Diffusion Pipeline

In [5]:
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16
)
pipe = pipe.to(device)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


## # Gradio block

In [18]:

selected_pixels = [] # store the pixels from the image on trigger of event

with gr.Blocks() as webApp:
    # Todo: prompt to take user input
    with gr.Row():
        input_img = gr.Image(label = "Input Image")
        mask_img = gr.Image(label = "mask")
        output_img = gr.Image(label="Output Image")

    with gr.Blocks():
        prompt_text = gr.Textbox(lines=1, label = "Prompt")

    with gr.Row():
        submit = gr.Button("submit")

    # Todo: Generate Mask
    # [Gathering Event Data](https://gradio.app/blocks-and-event-listeners/#gathering-event-data)
    def generate_mask(image, evt: gr.SelectData):

        selected_pixels.append(evt.index)

        predictor.set_image(image)
        # Points are input to the model in (x,y) format and come with labels 1 (foreground point) or 0 (background point).
        input_point = np.array(selected_pixels)
        input_label = np.ones(input_point.shape[0])

        masks, scores, logits = predictor.predict(
            point_coords=input_point,
            point_labels=input_label,
            multimask_output=False, # When False, it will return a single mask.
        )
        # this will return: masks.shape  # (number_of_masks) x H x W

        # convert mask to PIL image
        masks = np.logical_not(masks) # if we want to change background and not foreground
        mask = Image.fromarray(masks[0, : , :])

        return mask

    # Todo: Trigger event added below


    # Todo: Text-Guided Image Inpainting [StableDiffusionInpaintPipeline](https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/inpaint)
    def SD_inpaint(image, mask, prompt):
        image = Image.fromarray(image)
        mask = Image.fromarray(mask)

        image = image.resize((512, 512))
        mask = mask.resize((512,512))

        output_image = pipe(prompt=prompt, image=image, mask_image=mask).images[0]

        return output_image

    # from the input image create a trigger to generate mask
    input_img.select(generate_mask, [input_img], [mask_img])
    submit.click(
        fn=SD_inpaint,
        inputs=[input_img, mask_img, prompt_text],
        outputs=[output_img],
        api_name="submit"
    )


In [19]:
webApp.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>